# GVC participation rate

This notebook computes the trade-based and production-based GVC participation rates at the country-level and the 5-, 15-, and 35-sector levels. It uses the trade accounting framework of Borin and Mancini (2019). Results are saved as `gvcp.parquet` in `data/`.

The trade-based GVC participation rate is defined as
```
GVCP_trade_f = (REX1 + REX2 + REX3 + REF1 + REF2) / Exports
GVCP_trade_f = (FVA + PDC1 + PDC2) / Exports
GVCP_trade = GVCP_trade_f + GVCP_trade_b
```
where sectors are broken down by export sectors.

The production-based GVC participation rate, meanwhile, is defined as
```
GVCP_prod = (DAVAX2 + REX1 + REX2 + REX3 + REF1 + REF2) / va
```
where sectors are broken down by origin sectors.

In [1]:
import numpy as np
import pandas as pd
import duckdb

In [2]:
# ta = 'ta.parquet'
# summary = 'summary.parquet'
# output = 'gvcp.parquet'

# ta = 'ta62.parquet'
# summary = 'summary62.parquet'
# output = 'gvcp62.parquet'

ta = 'ta62-const.parquet'
summary = 'summary62-const.parquet'
output = 'gvcp62-const.parquet'

# ta = 'ta_jun2023.parquet'
# summary = 'summary_jun2023.parquet'
# output = 'gvcp_jun2023.parquet'

## Load and process data

### Breakdown by export sectors

In [5]:
df_es = duckdb.sql(
    f"""
    (SELECT t, s, 0 AS agg, 0 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/{ta}') WHERE breakdown='none'
    GROUP BY t, s
    ORDER BY t, s)

    UNION ALL

    (SELECT t, s, 5 AS agg, i5 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/{ta}') WHERE breakdown='es'
    GROUP BY t, s, i5
    ORDER BY t, s, i5)

    UNION ALL
    
    (SELECT t, s, 15 AS agg, i15 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/{ta}') WHERE breakdown='es'
    GROUP BY t, s, i15
    ORDER BY t, s, i15)
    
    UNION ALL
    
    (SELECT t, s, 35 AS agg, i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/{ta}') WHERE breakdown='es'
    GROUP BY t, s, i
    ORDER BY t, s, i)
    """
).df()

df_es['GVC_trade_f'] = df_es['REX1'] + df_es['REX2'] + df_es['REX3'] + df_es['REF1'] + df_es['REF2']
df_es['GVC_trade_b'] = df_es['FVA'] + df_es['PDC1'] + df_es['PDC2']
df_es['GVC_trade'] = df_es['GVC_trade_f'] + df_es['GVC_trade_b']
df_es['t'] = df_es['t'].astype(int)
df_es = df_es[['t', 's', 'agg', 'i', 'Exports', 'GVC_trade_f', 'GVC_trade_b', 'GVC_trade']]

### Breakdown by origin sectors

In [6]:
df_os = duckdb.sql(
    f"""
    (SELECT t, s, 0 AS agg, 0 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/{ta}') WHERE breakdown='none'
    GROUP BY t, s
    ORDER BY t, s)

    UNION ALL

    (SELECT t, s, 5 AS agg, i5 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/{ta}') WHERE breakdown='os'
    GROUP BY t, s, i5
    ORDER BY t, s, i5)

    UNION ALL
    
    (SELECT t, s, 15 AS agg, i15 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/{ta}') WHERE breakdown='os'
    GROUP BY t, s, i15
    ORDER BY t, s, i15)
    
    UNION ALL
    
    (SELECT t, s, 35 AS agg, i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/{ta}') WHERE breakdown='os'
    GROUP BY t, s, i
    ORDER BY t, s, i)
    """
).df()

df_os['GVC_prod'] = df_os['DAVAX2'] + df_os['REX1'] + df_os['REX2'] + df_os['REX3'] + df_os['REF1'] + df_os['REF2']
df_os['t'] = df_os['t'].astype(int)
df_os = df_os[['t', 's', 'agg', 'i', 'GVC_prod']]

### Value added

In [9]:
va = duckdb.sql(
    f"""
    (SELECT t, s, 0 AS agg, 0 AS i, SUM(va) AS va, 
    FROM read_parquet('../data/{summary}')
    GROUP BY t, s
    ORDER BY t, s)

    UNION ALL

    (SELECT t, s, 5 AS agg, i5 AS i, SUM(va) AS va, 
    FROM read_parquet('../data/{summary}')
    GROUP BY t, s, i5
    ORDER BY t, s, i5)

    UNION ALL

    (SELECT t, s, 15 AS agg, i15 AS i, SUM(va) AS va, 
    FROM read_parquet('../data/{summary}')
    GROUP BY t, s, i15
    ORDER BY t, s, i15)

    UNION ALL

    (SELECT t, s, 35 AS agg, i, SUM(va) AS va, 
    FROM read_parquet('../data/{summary}')
    GROUP BY t, s, i
    ORDER BY t, s, i)
    """
).df()

In [10]:
df_os = pd.merge(df_os, va)

## Consolidate and save

In [11]:
df = pd.merge(df_es, df_os)
df['GVCP_trade_f'] = df['GVC_trade_f'] / df['Exports']
df['GVCP_trade_b'] = df['GVC_trade_b'] / df['Exports']
df['GVCP_trade'] = df['GVC_trade'] / df['Exports']
df['GVCP_prod'] = df['GVC_prod'] / df['va']

df = df[[
    't', 's', 'agg', 'i', 'Exports', 'va',
    'GVC_trade_f', 'GVC_trade_b', 'GVC_trade', 'GVC_prod',
    'GVCP_trade_f', 'GVCP_trade_b', 'GVCP_trade', 'GVCP_prod'
]]

df.to_parquet(f'../data/{output}', index=False)

In [12]:
df

,t,s,agg,i,Exports,va,GVC_trade_f,GVC_trade_b,GVC_trade,GVC_prod,GVCP_trade_f,GVCP_trade_b,GVCP_trade,GVCP_prod
0,2007,1,0,0,232183.101521,1.161534e+06,59793.506013,33487.699426,93281.205439,160629.405129,0.257527,0.144230,0.401757,0.138291
1,2007,2,0,0,179044.726728,3.624658e+05,34738.021510,60372.927412,95110.948922,75311.345205,0.194019,0.337195,0.531213,0.207775
2,2007,3,0,0,302596.350948,4.580332e+05,53065.626981,124180.277742,177245.904723,115543.125495,0.175368,0.410383,0.585750,0.252259
3,2007,4,0,0,19596.166770,4.558134e+04,3253.156252,7916.892412,11170.048664,7979.185565,0.166010,0.404002,0.570012,0.175054
4,2007,5,0,0,255774.704147,1.898964e+06,54448.717715,25527.609337,79976.327052,147485.210075,0.212878,0.099805,0.312683,0.077666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56443,2022,63,35,31,15839.569786,4.939165e+05,3469.551685,2353.121939,5822.673624,14053.710945,0.219043,0.148560,0.367603,0.028454
56444,2022,63,35,32,11741.929403,3.417807e+05,1326.354642,1557.638915,2883.993557,6701.458875,0.112959,0.132656,0.245615,0.019607
56445,2022,63,35,33,10791.119652,2.469475e+05,719.960340,2242.282068,2962.242408,6924.245862,0.066718,0.207790,0.274507,0.028039
56446,2022,63,35,34,14521.176786,1.797342e+05,2409.412247,3326.134559,5735.546806,19882.189022,0.165924,0.229054,0.394978,0.110620
